# Metro transit information 

Using the metro API, this notebook gets data on travel times to Hattie Mae White on board meeting days. 

In [ ]:
!pip install airtable

In [360]:
import numpy as np
import pandas as pd
import requests
from airtable import airtable
import time
import dateutil
from google.colab import drive

## Testing things

It's been a while since I've worked with an API, so let's just make sure some things are right

In [ ]:
# Let's get all the information on the one route I ride the most often
response = requests.get("https://api.ridemetro.org/data/Routes?$filter=RouteName eq '004'&subscription-key=4e7902d1a2ae42df978fe20387049854")
print(response.status_code)

200


In [ ]:
response.json()

{'@odata.context': 'http://api.transitiq.com/ODataService.svc/$metadata#Routes',
 'value': [{'AgencyAbbreviation': 'RideMetro',
   'DataSource': 'Transit_5',
   'FinalStop0Id': None,
   'FinalStop1Id': None,
   'LongName': 'Beechnut',
   'RouteId': 'Ho414_4620_41894',
   'RouteName': '004',
   'RouteType': 'Bus'}]}

In [ ]:
# Now let's get all the information we can about this route. This is literally all the stops! 
response = requests.get("https://api.ridemetro.org/data/Routes('Ho414_4620_41894')?subscription-key=4e7902d1a2ae42df978fe20387049854&$expand=Stops")
print(response.status_code)

200


In [ ]:
response.json()

In [ ]:
# Let's get all the vehicles
response = requests.get("https://api.ridemetro.org/data/Vehicles?subscription-key=4e7902d1a2ae42df978fe20387049854")
print(response.status_code)


200


In [ ]:
response.json()

## Exploring bus stops around Hattie Mae White

Let's start with how accessible Hattie Mae White is by public transit: Let's see what stops are in the area

In [ ]:
# Hattie Mae White coordinatess

HMW_lat = 29.802759908899148
HMW_long = -95.45410037006431

key = "4e7902d1a2ae42df978fe20387049854"

In [ ]:
HMW_stops = requests.get(f"https://api.ridemetro.org/data/GeoAreas('{HMW_lat}|{HMW_long}|0.3')/Stops?subscription-key={key}")
print(HMW_stops.status_code)

200


In [ ]:
HMW_stops.json()

## Linking airtable data

Now, we'll calculate the distance from each school to HMW. To do this, we'll import information from the airtable base where I am putting all the information 

In [ ]:
airtable_api_key = "keyOZN2sN8u2NhZVY"
airtable_base_id = "appxpHxQKYCeuAa46"

In [ ]:
# taken from https://community.airtable.com/t/how-to-get-more-than-100-rows-using-airtable-api-in-python-using-offest/30536/2

global offset
offset = '0'
t = []

while True :
    url = f"https://api.airtable.com/v0/{airtable_base_id}/School%20Information"
    querystring = {
        # "view":"Grid View",
        "api_key": airtable_api_key,
        "offset": offset}

    try :
        response= requests.get(url, params=querystring)
        req = response.json()
        page = pd.DataFrame(req, columns = req.keys())
        page = pd.concat([page.drop(['records'], axis=1), page['records'].apply(pd.Series)], axis=1)
        page = pd.concat([page.drop(['fields'], axis=1), page['fields'].apply(pd.Series)], axis=1).fillna('')

        t.append(page)
        print(records[0]['id'] , len(records))
    
        try : 
            offset = req['offset']
            print(offset)
            
        except Exception as ex:
            print(ex , offset)
            break
    
    except error as e:
        print("hi")

schools = pd.concat(t, axis=0).reset_index()

# take the columns we want and rename them
schools = schools[['School Name', 'Latitude [Public School] 2019-20', 'Longitude [Public School] 2019-20', 'School ID - NCES Assigned [Public School] Latest available year']]
schools.columns = ['name', 'lat', 'long', 'id']

schools.head(20)

reciHyqBq9uMgr37c 80
itryrZzCskZgswaT0/recLHuirfHVVPFD5m
reciHyqBq9uMgr37c 80
itryrZzCskZgswaT0/rechwX48Ju9zvX1zZ
reciHyqBq9uMgr37c 80
'offset' itryrZzCskZgswaT0/rechwX48Ju9zvX1zZ


,name,lat,long,id
0,GRISSOM EL,29.620428,-95.455033,482364002472
1,KING EARLY CHILDHOOD CTR,29.614025,-95.436000,482364010746
2,KOLTER EL,29.712887,-95.455950,482364002510
3,HOBBY EL,29.635314,-95.436981,482364002490
4,WELCH MIDDLE,29.653218,-95.529156,482364005501
5,HARRIS J R EL,29.718712,-95.277378,482364002478
6,BELLAIRE H S,29.692163,-95.469159,482364002397
7,MADISON H S,29.629169,-95.435617,482364002530
8,WOODSON SCHOOL,29.643053,-95.366681,482364002417
9,MICKEY LELAND COLLEGE PREP ACAD FOR YOUNG MEN,29.776713,-95.335360,482364012618


In [ ]:
# now, let's take a sample school and calculat the route! 
nrow = 2

s_lat = schools.iloc[nrow, 1]
s_long = schools.iloc[nrow, 2]

print(schools.iloc[nrow, 0])
print(s_lat, s_long)

29.67397345835996, -95.46738418104776


KOLTER EL
29.712887 -95.45595


In [ ]:
trip = requests.get(f"https://api.ridemetro.org/data/CalculateItineraryByPoints?lat1={s_lat}&lon1={s_long}&lat2={HMW_lat}&lon2={HMW_long}&$orderby=EndTime&$expand=Legs&subscription-key={key}")
print(trip.status_code)

200


In [ ]:
pd.DataFrame(trip.json())

,@odata.context,value
0,http://api.transitiq.com/ODataService.svc/$met...,{'ItineraryId': 'f04b4da1-c975-4358-9b1d-5d23b...
1,http://api.transitiq.com/ODataService.svc/$met...,{'ItineraryId': 'acc34e88-d91c-4b8d-bd88-262e5...


In [ ]:
trip_df = trip['value'].apply(pd.Series)
trip_df['school'] = schools.iloc[nrow, 0]
trip_df['school_id'] = schools.iloc[nrow, 3]
trip_df.head()

,ItineraryId,Created,Kml,StartTime,EndTime,AdjustedStartTime,AdjustedEndTime,StartStopName,EndStopName,StartStopId,EndStopId,StartAddress,EndAddress,TransferCount,TravelTypes,WalkDistance,RequestId,StartLon,StartLat,EndLon,EndLat,LocalTimeOffset,Legs,school,school_id
0,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,2021-11-19T20:54:17.6687933Z,http://api.transitiq.com/ODataService.svc/GetI...,2021-11-19T21:32:29Z,2021-11-19T23:14:48Z,2021-11-19T21:32:29Z,2021-11-19T23:14:48Z,S Post Oak Rd @ Heatherbrook Dr,Hempstead Rd @ Post Oak Rd,Ho414_4620_5256,Ho414_4620_11053,None,None,2,"Walk,Bus",1.247141,0388707b-346d-4211-b1b3-3d705fe18a26,-95.455033,29.620428,-95.4541,29.80276,-5,"[{'ItineraryLegId': 1108178896, 'AgencyAbbrevi...",KOLTER EL,482364002510
1,8bbdbabb-ad36-4f4d-b341-89ef431dcaf8,2021-11-19T20:54:17.6691807Z,http://api.transitiq.com/ODataService.svc/GetI...,2021-11-19T21:32:29Z,2021-11-19T23:23:32Z,2021-11-19T21:32:29Z,2021-11-19T23:23:32Z,S Post Oak Rd @ Heatherbrook Dr,Hempstead Rd @ W Loop N,Ho414_4620_5256,Ho414_4620_8896,None,None,2,"Walk,Bus",1.629523,0388707b-346d-4211-b1b3-3d705fe18a26,-95.455033,29.620428,-95.4541,29.80276,-5,"[{'ItineraryLegId': 1108179896, 'AgencyAbbrevi...",KOLTER EL,482364002510


In [ ]:
legs = trip_df['Legs'].apply(pd.Series)

In [ ]:
legs.iloc[0].apply(pd.Series)

,ItineraryLegId,AgencyAbbreviation,AgencyName,Created,StopName,StopId,RouteId,RouteName,DirectionId,DirectionVariantId,DirectionName,RouteDestinationName,StartTime,AdjustedStartTime,Duration,Length,Address,ShapeUri,TransportType,Ordinal,ItineraryId,DataSource,Lat,Lon,TripId,TripHeadsign,DelayFromScheduleInSeconds
0,1108178896,None,None,2021-11-19T20:54:17.6687139Z,,,,,,,,,2021-11-19T21:31:29Z,2021-11-19T21:31:29Z,979,0.843312,,,Walk,0,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.620428,-95.455033,,,NaN
1,1108178897,RideMetro,Houston,2021-11-19T20:54:17.6687662Z,S Post Oak Rd @ Heatherbrook Dr,Ho414_4620_5256,Ho414_4620_41922,049,Ho414_4620_41922_0,Ho414_4620_41922_0_1,NORTHWEST TC,NORTHWEST TC,2021-11-19T21:48:48Z,2021-11-19T21:48:48Z,4152,11.086886,,,Bus,1,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.622834,-95.464965,Ho414_4620_9267310,NORTHWEST TC,0.0
2,1108178898,RideMetro,Houston,2021-11-19T20:54:17.668788Z,79 - NORTHWEST BAY M,Ho414_4620_79_M,Ho414_4620_41928,058,Ho414_4620_41928_1,Ho414_4620_41928_1_0,WEST BELT,WEST BELT,2021-11-19T23:00:00Z,2021-11-19T23:00:00Z,360,1.104310,,,Bus,3,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.783070,-95.455460,Ho414_4620_9269230,WEST BELT,0.0
3,1108178899,None,None,2021-11-19T20:54:17.6687885Z,Hempstead Rd @ Post Oak Rd,Ho414_4620_11053,,,,,,,2021-11-19T23:07:00Z,2021-11-19T23:07:00Z,468,0.403830,,,Walk,4,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.799026,-95.456516,,,NaN
4,1108178900,None,None,2021-11-19T20:54:17.6687885Z,,,,,,,,,2021-11-19T23:07:00Z,2021-11-19T23:07:00Z,468,0.403830,,,Arrived,5,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.802760,-95.454100,,,NaN


In [ ]:
schools.head(20)

,name,lat,long,id
0,GRISSOM EL,29.620428,-95.455033,482364002472
1,KING EARLY CHILDHOOD CTR,29.614025,-95.436000,482364010746
2,KOLTER EL,29.712887,-95.455950,482364002510
3,HOBBY EL,29.635314,-95.436981,482364002490
4,WELCH MIDDLE,29.653218,-95.529156,482364005501
5,HARRIS J R EL,29.718712,-95.277378,482364002478
6,BELLAIRE H S,29.692163,-95.469159,482364002397
7,MADISON H S,29.629169,-95.435617,482364002530
8,WOODSON SCHOOL,29.643053,-95.366681,482364002417
9,MICKEY LELAND COLLEGE PREP ACAD FOR YOUNG MEN,29.776713,-95.335360,482364012618


## Running it on all the schools 

Now, let's loop through all the schools and put all the routes together 

In [ ]:
routelist = []

for nrow in range(0, len(schools)): 

  s_lat = schools.iloc[nrow, 1]
  s_long = schools.iloc[nrow, 2]

  # get the trip information for that school
  trip = requests.get(f"https://api.ridemetro.org/data/CalculateItineraryByPoints?lat1={s_lat}&lon1={s_long}&lat2={HMW_lat}&lon2={HMW_long}&$orderby=EndTime&$expand=Legs&subscription-key={key}")

  trip1 = pd.DataFrame(trip.json())

  trip_df = trip1['value'].apply(pd.Series)
  trip_df['school'] = schools.iloc[nrow, 0]
  trip_df['school_id'] = schools.iloc[nrow, 3]

  routelist.append(trip_df)
  print(f"Done with school #{nrow}: {schools.iloc[nrow, 0]}")

  time.sleep(2)   

Done with school #0
Done with school #1
Done with school #2
Done with school #3
Done with school #4
Done with school #5
Done with school #6
Done with school #7
Done with school #8
Done with school #9
Done with school #10
Done with school #11
Done with school #12
Done with school #13
Done with school #14
Done with school #15
Done with school #16
Done with school #17
Done with school #18
Done with school #19
Done with school #20
Done with school #21
Done with school #22
Done with school #23
Done with school #24
Done with school #25
Done with school #26
Done with school #27
Done with school #28
Done with school #29
Done with school #30
Done with school #31
Done with school #32
Done with school #33
Done with school #34
Done with school #35
Done with school #36
Done with school #37
Done with school #38
Done with school #39
Done with school #40
Done with school #41
Done with school #42
Done with school #43
Done with school #44
Done with school #45
Done with school #46
Done with school #47
Do

In [373]:
routes = pd.concat(routelist, ignore_index = True)
routes['AdjustedStartTime'] = routes.AdjustedStartTime.apply(dateutil.parser.isoparse)
routes['AdjustedEndTime'] = routes.AdjustedEndTime.apply(dateutil.parser.isoparse)
routes['length'] = routes['AdjustedEndTime'] - routes['AdjustedStartTime']

all_legs = routes['Legs'].apply(pd.Series)
leglist = []

for route in range(0, len(all_legs)): 

  temp = all_legs.iloc[route, :].apply(pd.Series)
  leglist.append(temp)

legs = pd.concat(leglist)
print(legs.shape)
legs = legs[~np.isnan(legs['Ordinal'])]
legs.shape

(12410, 28)


(8073, 28)

In [337]:
print(f"Number of schools: {len(schools)}")
print(f"Number of routes: {len(routes)}")
print(f"Number of legs: {len(legs)}")

Number of schools: 280
Number of routes: 1241
Number of legs: 8073


In [ ]:
legs.sort_values(['ItineraryId', 'Ordinal'], inplace = True)
legs.head()

In [377]:
mintrans = routes[routes.TravelTypes != "Walk"].sort_values('TransferCount').drop_duplicates(subset = 'school_id', keep = 'first')
mintrans.TransferCount.value_counts()

2    164
3     79
1     35
4      1
Name: TransferCount, dtype: int64

In [378]:
mintrans.length.describe()

count                          279
mean     0 days 01:20:12.820788530
std      0 days 00:28:27.858972493
min                0 days 00:15:58
25%                0 days 01:01:52
50%                0 days 01:18:38
75%         0 days 01:37:44.500000
max                0 days 03:15:26
Name: length, dtype: object

In [376]:
routes.length.describe()

count                         1241
mean     0 days 01:29:57.339242546
std      0 days 00:29:12.139752715
min                0 days 00:00:31
25%                0 days 01:10:36
50%                0 days 01:27:10
75%                0 days 01:49:22
max                0 days 03:47:45
Name: length, dtype: object

In [379]:
minlen = routes.sort_values('length').drop_duplicates(subset = 'school_id', keep = 'first')
minlen.length.describe()


count                          280
mean     0 days 01:13:08.482142857
std      0 days 00:26:30.237352846
min                0 days 00:00:31
25%         0 days 00:54:36.250000
50%                0 days 01:12:16
75%         0 days 01:30:28.250000
max                0 days 02:49:53
Name: length, dtype: object

In [383]:
sum(minlen.length > datetime.timedelta(hours = 1))

187